# MT-Bench Multi-Turn Dialogue Evaluation

## Purpose
This notebook implements **MT-Bench (FastChat)** evaluation to assess multi-turn dialogue quality of our fine-tuned LLaMA-2-7B model. This is the second critical benchmark required by [Assignment 7](../../tasks/Assignment7.md) to demonstrate the effectiveness of our LoRA fine-tuning approach on conversational capabilities.

## Evaluation Context
Following the complete fine-tuning pipeline:

1. **[Baseline Model Evaluation](2_baseline_model.ipynb)** - Established baseline performance metrics
2. **[Fine-tuning Process](3_finetuning.ipynb)** - LoRA fine-tuning on Dolly-15K dataset  
3. **[Fine-tuned Model Testing](4_finetune_model.ipynb)** - Inference and testing with fine-tuned model
4. **[AlpacaEval 2 Benchmark](5_benchmark_alpaca_eval.ipynb)** - Instruction-following quality assessment
5. **This Notebook** - MT-Bench multi-turn dialogue evaluation
6. **Final Report** - Comprehensive analysis and results

## MT-Bench Framework
- **Repository**: https://github.com/lm-sys/FastChat
- **Purpose**: Multi-turn dialogue quality evaluation
- **Dataset**: 80 multi-turn conversations across 8 categories
- **Method**: GPT-4 as judge to evaluate conversation quality, helpfulness, and consistency
- **Metrics**: Overall score, category-specific scores, consistency across turns

## Expected Improvements
After LoRA fine-tuning on Dolly-15K, we expect to see:
- **Better conversation flow** - More coherent multi-turn interactions
- **Improved consistency** - Maintains context across conversation turns
- **Enhanced helpfulness** - More useful and relevant responses in conversations
- **Better dialogue structure** - Proper conversation formatting and flow

## Technical Implementation
- **Model**: Fine-tuned LLaMA-2-7B with LoRA adapters
- **Process**:
  1. Load our fine-tuned model
  2. Run MT-Bench evaluation dataset (80 multi-turn conversations)
  3. Use GPT-4 as automated judge to evaluate conversation quality
  4. Calculate overall and category-specific scores
- **Comparison**: Fine-tuned vs. baseline model performance on multi-turn dialogue

## Workflow
1. **Load fine-tuned model** from saved LoRA adapters
2. **Load MT-Bench dataset** (80 multi-turn conversations)
3. **Run evaluation** using MT-Bench framework
4. **Generate scores** for overall and category-specific performance
5. **Compare results** with baseline model performance
6. **Document metrics** for final report

## Success Criteria
- **Higher overall score** than baseline model on MT-Bench
- **Improved category scores** across different conversation types
- **Better consistency** in multi-turn conversations
- **Clear evidence** of enhanced conversational capabilities

## MT-Bench Categories
The evaluation covers 8 conversation categories:
1. **Writing** - Creative and technical writing tasks
2. **Roleplay** - Character and scenario-based conversations
3. **Reasoning** - Logical and mathematical reasoning
4. **Math** - Mathematical problem solving
5. **Coding** - Programming and code-related discussions
6. **Extraction** - Information extraction tasks
7. **STEM** - Science, technology, engineering, math
8. **Humanities** - Social sciences, history, philosophy

---
**Note**: This evaluation is essential for demonstrating that our LoRA fine-tuning approach successfully improves the model's multi-turn conversational capabilities, complementing the instruction-following improvements shown in AlpacaEval 2.


In [ ]:
!pip install -q git+https://github.com/lm-sys/FastChat.git
!git clone https://github.com/lm-sys/FastChat.git
%cd FastChat
!pip install -e ".[model_worker,llm_judge]"
!pip install -U transformers peft bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel
import os
from google.colab import drive


In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
#Load the fine-tuned model
model_id = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

adapter_path = "/content/drive/MyDrive/LLaMA2-Dolly-Training/results/final_lora_adapter"
print(f"Loading Fine-Tuned LoRA adapter from: {adapter_path}...")
model = PeftModel.from_pretrained(base_model, adapter_path)

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(model)


In [ ]:
merged_model_path = "/content/drive/MyDrive/LLaMA2-Dolly-Training/models/Llama-2-7b-hf-dolly-merged"
model = model.merge_and_unload()
os.makedirs(merged_model_path, exist_ok=True)
model.save_pretrained(merged_model_path)
tokenizer = AutoTokenizer.from_pretrained(adapter_path)
tokenizer.save_pretrained(merged_model_path)
print(model)

In [ ]:
output_dir = "/content/drive/MyDrive/LLaMA2-Dolly-Training/outputs"
base_model_path = "/content/drive/MyDrive/LLaMA2-Dolly-Training/models/Llama-2-7b-hf"
merged_model_path = "/content/drive/MyDrive/LLaMA2-Dolly-Training/models/Llama-2-7b-hf-dolly-merged"

model_id_finetuned = "llama-2-7b-dolly-qlora"
default_answer_file_finetuned = f"/content/FastChat/fastchat/llm_judge/data/mt_bench/model_answer/{model_id_finetuned}.jsonl"
gdrive_answer_file_finetuned = f"{output_dir}/{model_id_finetuned}_mt_bench_answers.jsonl"

model_id_baseline = "llama-2-7b-hf-baseline"
default_answer_file_baseline = f"/content/FastChat/fastchat/llm_judge/data/mt_bench/model_answer/{model_id_baseline}.jsonl"
gdrive_answer_file_baseline = f"{output_dir}/{model_id_baseline}_mt_bench_answers.jsonl"

os.makedirs(output_dir, exist_ok=True)
fastchat_judge_dir = "/content/FastChat/fastchat/llm_judge"
os.chdir(fastchat_judge_dir)




In [ ]:

!echo "Generating MT-Bench answers for Fine-Tuned Model (${model_id_finetuned})..."
!python3 gen_model_answer.py --model-path "{merged_model_path}" --max-new-token 2048 --model-id "{model_id_finetuned}" --answer-file "{gdrive_answer_file_finetuned}"
!echo "✓ Fine-tuned answers generated to default path: {default_answer_file_finetuned}"







In [ ]:
print("\nGenerating MT-Bench answers for Baseline Model...")
!python3 gen_model_answer.py \
    --model-path "{base_model_path}" \
    --model-id "{model_id_baseline}" \
    --answer-file "{gdrive_answer_file_baseline}" \
    --max-new-token 2048

In [ ]:
print(gdrive_answer_file_baseline)
print(gdrive_answer_file_finetuned)